**Pyeeg has been installed**

In [ ]:
pip install git+https://github.com/forrestbao/pyeeg.git

  Cloning https://github.com/forrestbao/pyeeg.git to /tmp/pip-req-build-zq1onecj
  Running command git clone --filter=blob:none --quiet https://github.com/forrestbao/pyeeg.git /tmp/pip-req-build-zq1onecj
  Resolved https://github.com/forrestbao/pyeeg.git to commit a6c18bb093e4748f9d9c208535a6ae024a0802b8
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install pyeeg

**Imported the necessery library packages**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import seaborn as sns
import plotly.express as px

%matplotlib inline

from sklearn.preprocessing import normalize

import math
import os
import time
import pickle as pickle

In [ ]:
import pyeeg as pe

**Here is the code for mounting google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path_to_dataset_2 = '/content/drive/MyDrive/EEG/data_preprocessed_python/' # Location of dat file of DEAP dataset
path_to_dataset = '/content/drive/MyDrive/EEG/numPy/' # Where to save preprocessed npy files

# channel = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40] #40 Channels chosen to fit Emotiv Epoch+

channel = list(range(40))  # Channels 0-39 to match the data array
band = [4, 8, 12, 16, 25, 45]  # 5 bands
window_size = 256  # Averaging band power of 2 sec
step_size = 16  # Each 0.125 sec update once
sample_rate = 128  # Sampling rate of 128 Hz

subjectList = [f"{i:02d}" for i in range(1, 33)]  # Create subject list with zero-padded numbers

def FFT_Processing(sub, channel, band, window_size, step_size, sample_rate):
    '''
    arguments:  string subject
                list channel indices
                list band
                int window size for FFT
                int step size for FFT
                int sample rate for FFT
    return:     void
    '''
    meta = []
    with open(path_to_dataset_2 + 's' + sub + '.dat', 'rb') as file:
        subject = pickle.load(file, encoding='latin1')  # Resolve Python 2 data issue

        for i in range(40):  # Loop over 40 trials
            data = subject["data"][i]
            labels = subject["labels"][i]
            start = 0

            while start + window_size < data.shape[1]:
                meta_data = []  # Meta vector for analysis
                for j in channel:
                    X = data[j][start: start + window_size]  # Slice raw data
                    Y = pe.bin_power(X, band, sample_rate)  # FFT for channel `j`
                    meta_data += list(Y[0])  # Append the band power data

                # Ensure `labels` is a 1D array or list
                meta_array = [np.array(meta_data), np.array(labels).flatten()]
                meta.append(meta_array)
                start += step_size

        meta = np.array(meta, dtype=object)  # Use dtype=object to handle variable-length arrays
        np.save(path_to_dataset + 's' + sub, meta, allow_pickle=True, fix_imports=True)

for subjects in subjectList:
    FFT_Processing(subjects, channel, band, window_size, step_size, sample_rate)

In [ ]:
import numpy as np
import pandas as pd

# Initialize the data and label lists
data_training = []
label_training = []
data_testing = []
label_testing = []

# Loop through each subject (01 to 32)
for subject_id in range(1, 33):
    # Format the subject number and load the corresponding file
    subject_file = f"{path_to_dataset}s{subject_id:02d}.npy"

    # Load the subject's data
    with open(subject_file, 'rb') as file:
        sub = np.load(file, allow_pickle=True)
        for i in range(sub.shape[0]):  # Iterate through the rows
            if i % 4 == 0:  # 25% data for testing
                data_testing.append(sub[i][0])
                label_testing.append(sub[i][1])
            else:  # 75% data for training
                data_training.append(sub[i][0])
                label_training.append(sub[i][1])

# Convert data and labels to NumPy arrays
data_training = np.array(data_training)
label_training = np.array(label_training)
data_testing = np.array(data_testing)
label_testing = np.array(label_testing)

# Define column names for features
n_features = data_training.shape[1]
column_names = [f'C{i+1}B{j+1}' for i in range(n_features // 5) for j in range(5)]

# Create the training DataFrame
train_df = pd.DataFrame(data_training, columns=column_names)
train_df['Valence'], train_df['Arousal'], train_df['Domain'], train_df['Like'] = label_training.T

# Create the testing DataFrame
test_column_names = [f'C{i+1}B{j+1}' for i in range(data_testing.shape[1] // 5) for j in range(5)]
test_df = pd.DataFrame(data_testing, columns=test_column_names)
test_df['Valence'], test_df['Arousal'], test_df['Domain'], test_df['Like'] = label_testing.T

# Optionally save to CSV
# save_path = "/path/to/save/"
# train_df.to_csv(save_path + "train_data.csv", index=False)
# test_df.to_csv(save_path + "test_data.csv", index=False)

print("Training and testing datasets created successfully.")